<img src="https://github.com/denisabrantes/pdk-use-cases/blob/main/images/hpe_logo.png?raw=true" alt="HPE Logo" width="250" style="align:left;margin-left:0;"/>

<h1>Object Detection Prediction from KServe InferenceService</h1>

<b>Date: 01/02/24</b><br/>
<b>Version: 0.1</b><br/>
<b>Authors: HPE AI At Scale SE Team</b>

<img src="https://github.com/denisabrantes/pdk-use-cases/blob/main/images/platform_step0.png?raw=true" alt="Enterprise Machine Learning platform architecture" width="850">

<h3>Import modules and define functions</h3>
The cells below imports all modules and libraries required to run the demo.

In [1]:
# !pip install -q ipywidgets
# !pip install -q scikit-image

In [2]:
import sys
import glob
import base64
import json
import requests
import matplotlib.pyplot as plt

from skimage import io
from PIL import Image, ImageDraw
from ipywidgets import interact, interactive
import ipywidgets as widgets
import io

from tqdm import tqdm
from multiprocessing import Pool

<h2>Part 1: Use this code to load images from a folder and generate predictions</h2>

<h4>Step 1: Setting up connection details to KServe</h4>

In [26]:
# http://detection-deploy.pdk.192.168.11.103.sslip.io/v1/models
endpoint_name = "detection-deploy"
model_name ="object-detection"
ingress_host = "192.168.11.103"
ingress_port = "80"
service_hostname = "detection-deploy.pdk.192.168.11.103.sslip.io"

In [10]:
# !curl -v http://detection-deploy.pdk.192.168.11.103.sslip.io/v1/models/object-detection:predict

<h4>Step 2: Request prediction from KServe InferenceService</h4>

In [11]:
def plot_pred(im,pred_d,thres=0.15):
    draw = ImageDraw.Draw(im)
    try:
        for pred in pred_d['predictions'][0]:
            assert len(list(pred.keys())) == 2
            cl_name = list(pred.keys())[0]
            bboxes = pred[cl_name]
            if pred['score'] > thres:
                draw.rectangle([bboxes[0],bboxes[1],bboxes[2],bboxes[3]],outline=(255,0,0),fill=None,width=1)
                draw.text([bboxes[0],bboxes[1]-10],"{} :{:.2f}".format(cl_name,pred['score']),fill=(250,0,0))
        plt.figure(figsize=(8,8))
    except Exception as e:
        print(e)
        pass
    plt.imshow(im)
    plt.show()
    return im

In [27]:
def predict(args):
    image= Image.open(args[0])
    with io.BytesIO() as buffer:
        image.save(buffer, format='JPEG')
        image_bytes = buffer.getvalue()
    image_64_encode = base64.b64encode(image_bytes)
    bytes_array = image_64_encode.decode("utf-8")
    
    # Format the request in json
    request = {
      "instances":[
        {
          "data": bytes_array
        }
      ]
    }
    ingress_host = args[1]
    ingress_port = args[2]
    model_name = args[3]
    service_hostname = args[4]

    url = str("http://") + str(ingress_host) + ":" + str(ingress_port) + "/v1/models/" + str(model_name) + ":predict"
    # url = str("http://detection-deploy.pdk.") + str(ingress_host)+ "/v1/models/" + str(model_name) + ":predict"

    print(url)
    headers = {'Host': service_hostname, "Content-Type" : "application/json"}
    payload = json.dumps(request)

    response = requests.post(url, data=payload, headers=headers)
    print(response)
    return response.json()

In [13]:
def visualize(idx,thres=0.15):
    print(idx,thres)
    output = resps[idx]
    im = Image.open(imgs[idx])
    plot_pred(im,output,thres)

In [14]:
def run_apply_async_multiprocessing(func, argument_list, num_processes):
    pool = Pool(processes=num_processes)

    jobs = [pool.apply_async(func=func, args=(*argument,)) if isinstance(argument, tuple) else pool.apply_async(func=func, args=(argument,)) for argument in argument_list]
    pool.close()
    result_list_tqdm = []
    for job in tqdm(jobs):
        result_list_tqdm.append(job.get())

    return result_list_tqdm

In [16]:
# !curl -v http://detection-deploy.pdk.192.168.11.103.sslip.io/v1/models

In [17]:
images = r"./images/*.jpg"

In [18]:
imgs = [img for img in glob.glob(images, recursive=True)]

In [29]:
resps = run_apply_async_multiprocessing(predict,
                                        [[imgs[i],
                                        ingress_host,
                                        ingress_port,
                                        model_name,
                                        service_hostname] for i in range(len(imgs))],num_processes=4)

  0%|          | 0/1 [00:00<?, ?it/s]

http://192.168.11.103:80/v1/models/object-detection:predict
<Response [200]>


100%|██████████| 1/1 [00:02<00:00,  2.23s/it]


In [31]:
# !curl -v http://detection-deploy-predictor.pdk.192.168.11.103.sslip.io/v1/models/object-detection:predict

In [30]:
# predict([imgs[0],ingress_host,ingress_port,model_name,service_hostname])

In [ ]:
!curl -v http://192.168.11.103/v1/models/

In [32]:
interact(visualize, 
         idx=widgets.IntSlider(min=0, max=len(resps), step=1, value=0),
         thres = widgets.FloatSlider(min=0, max=1.0, step=0.1, value=0.00));

interactive(children=(IntSlider(value=0, description='idx', max=1), FloatSlider(value=0.0, description='thres'…

<h2>Part 2: Use this code to load the sample .json files and generate predictions</h2>

In [ ]:
with open("object_detection.json", "r") as objfile:
    imgfile = json.loads(objfile.read())

In [ ]:
jsonimg = imgfile["instances"][0]["data"]

In [ ]:
# Format the request in json
request = {
  "instances":[
    {
      "data": jsonimg
    }
  ]
}

url = str("http://") + str(ingress_host) + ":" + str(ingress_port) + "/v1/models/" + str(model_name) + ":predict"
headers = {'Host': service_hostname, "Content-Type" : "application/json"}
payload = json.dumps(request)

response = requests.post(url, data=payload, headers=headers)

In [ ]:
print(response.json())